In [ ]:
!(python3 train_ganomaly.py \
  \
  --epochs 1 \
  --batch_size 64 \
  --learning_rate 0.0002 \
  \
  --dataset_name mnist \
  --cache_path /tmp \
  --abnormal_class 2 \
  --image_size 32 \
  --image_channels 0 \
  --buffer_size 1000 \
  --shuffle y \
  --prefetch y \
  --random_brightness n \
  --random_crop n \
  --random_flip n \
  --repeat_dataset 0 \
  \
  --latent_size 100 \
  --n_extra_layers 0 \
  --n_filters 64 \
  --w_adv 1 \
  --w_rec 50 \
  --w_enc 1 \
  \
  --train_steps 1 \
  --eval_steps 1 \
  --log_level debug \
  --debug y \
  \
  --data_dir ./trainig/data \
  --model_dir ./trainig/model \
  --output_data_dir ./trainig/output \
 )

In [ ]:
from sagemaker.tensorflow import TensorFlow
import sagemaker
import subprocess
import os

In [ ]:
bucket = 'sagemaker-thesis-anomaly-detection'

artifacts_path = 'ganomaly/testing'
artifacts_path_s3 = 's3://{}/training/{}'.format(bucket, artifacts_path)

dataset_name = 'currently_unused'
dataset_path_s3 = 's3://{}/datasets/{}'.format(bucket, dataset_name)

In [ ]:
hyperparameters = {
    # training params
    'epochs': 1,
    'batch_size': 64,
    'learning_rate': 0.0002,

    # tf.data piepline params
    'dataset_name': 'mnist',
    'cache_path': '/tmp/tfdata',
    'abnormal_class': 2,  # only valid for mnist, fashion_mnist, cifar10, cifar100 and stl10
    'image_size': 32,
    'image_channels': 0,  # only valid for MVTec AD
    'buffer_size': 1000,
    'shuffle': True,
    'prefetch': True,
    'random_flip': False,
    'random_crop': False,
    'random_brightness': False,
    'repeat_dataset': 0,

    # model params
    'latent_size': 100,
    'n_filters': 64,
    'n_extra_layers': 0,
    'w_adv': 1,
    'w_rec': 50,
    'w_enc': 1,

    # debugging params
    #'train_steps': None,
    #'eval_steps': None,
    'log_level': 'info',
    'debug': False,

    # input/output dir params
    # they are set through env vars
    #'data_dir': './trainig/data',
    #'model_dir': './trainig/model',
    #'output_data_dir': './trainig/output'
}

In [ ]:
local_hyperparameters = hyperparameters.copy()
local_hyperparameters['epochs']      = 1
#local_hyperparameters['train_steps'] = 1
#local_hyperparameters['eval_steps']  = 1

estimator = TensorFlow(
    entry_point = 'train_ganomaly.py',
    source_dir = os.getcwd(),
    role = sagemaker.get_execution_role(),
    framework_version = '2.3.0',
    py_version = 'py37',
    hyperparameters = local_hyperparameters,
    train_instance_count = 1,
    train_instance_type = 'local' if subprocess.call('nvidia-smi') != 0 else 'local_gpu',
    #code_location = "file://" + artifacts_path_local,
    #output_path = "file://" + artifacts_path_local
    code_location = artifacts_path_s3,
    output_path = artifacts_path_s3
)

inputs = {
    'data_dir' : dataset_path_s3
}

estimator.fit(
    #inputs
)